In [1]:
import os
directory = 'models/simple-pytorch-model/1'
os.makedirs(directory, exist_ok=True)

In [2]:
import time
import numpy as np
import spacy
import torch
import torch.nn as nn
import torchtext
from torch.utils.data import DataLoader, Dataset, random_split
import torch.optim as optim
import torch.nn.functional as fn

In [3]:
import torch
from torch import nn
from torchvision import models


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = models.resnet50(weights="IMAGENET1K_V1")

    def forward(self, x):
        return self.model(x)

model = Model().eval().cuda()

In [4]:
class PyTorch_to_TorchScript(nn.Module):
    def __init__(self, my_model):
        super(PyTorch_to_TorchScript, self).__init__()
        self.model = my_model.model

    def forward(self, x):
        return self.model(x)

torchscript_model = PyTorch_to_TorchScript(model).eval().cuda()
traced_script_module = torch.jit.script(torchscript_model)
traced_script_module.save('models/simple-pytorch-model/1/model.pt')

In [5]:
config = """
name: "simple-pytorch-model"
platform: "pytorch_libtorch"
max_batch_size: 32
input [
{
name: "input__0"
data_type: TYPE_FP32
format: FORMAT_NCHW
dims: [3, 224, 224]
}
]
output [ 
{
name: "output__0"
data_type: TYPE_FP32
dims: [1000]
}
]
parameters: {
  key: "EXECUTION_ENV_PATH",
  value: {string_value: "D:/Abhi/COURSERA/Pytorch-practice/Classification/models/simple-pytorch-model/1/model.pt"}
}
"""

with open('models/simple-pytorch-model/config.pbtxt', 'w') as file:
    file.write(config)

In [34]:
!curl -v localhost:8000/v2/health/ready

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1:8000...
* Connected to localhost (127.0.0.1) port 8000 (#0)
> GET /v2/health/ready HTTP/1.1

> Host: localhost:8000

> User-Agent: curl/8.0.1

> Accept: */*

> 

< HTTP/1.1 200 OK

< Content-Length: 0

< Content-Type: text/plain

< 


  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
* Connection #0 to host localhost left intact


In [36]:
!curl -v localhost:8000/v2/models/simple-pytorch-model

{"error":"Request for unknown model: 'simple-pytorch-model' is not found"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1:8000...
* Connected to localhost (127.0.0.1) port 8000 (#0)
> GET /v2/models/simple-pytorch-model HTTP/1.1

> Host: localhost:8000

> User-Agent: curl/8.0.1

> Accept: */*

> 

< HTTP/1.1 400 Bad Request

< Content-Type: application/json

< Content-Length: 74

< 

{ [74 bytes data]

100    74  100    74    0     0  17366      0 --:--:-- --:--:-- --:--:-- 18500
* Connection #0 to host localhost left intact


In [27]:
import tritonclient.http as tritonhttpclient

In [36]:
VERBOSE = False
input_name = 'input__0'
input_shape = (1, 3, 224, 224)
input_dtype = 'FP32'
output_name = 'output__0'
model_name = 'simple-pytorch-model'
url = 'localhost:8000'
model_version = '1'

In [37]:
triton_client = tritonhttpclient.InferenceServerClient(url=url, verbose=VERBOSE)
model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)
model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)

InferenceServerException: [400] Request for unknown model: 'simple-pytorch-model' is not found